!pip install janus-core

# Nudge Elastic bands for lithium iron phosphate

The foundation potentials (called universal by some) provide an easy way to use MLIPs. In this tutorial we will see how well some of these models perform against each other and against the published DFT literature.

A top chart of main MLIPs is available

https://matbench-discovery.materialsproject.org/

References:

1. https://pubs.rsc.org/en/content/articlelanding/2011/ee/c1ee01782a
2. https://doi.org/10.1103/PhysRevApplied.7.034007
3. https://arxiv.org/abs/2401.00096
4. https://arxiv.org/abs/2402.03789

As a task we will determine the activation energies of Li diffusion along the [010] and [001] directions (or Paths b and c how we call them here) in lithium iron phosphate (LiFePO_4) a cathode material for lithium ion batteries.   
   
DFT references energies are: barrier heights: path b = 0.27 eV and path c = 2.5 eV. (see table 1 in https://doi.org/10.1039/C5TA05062F)

We will try CHGNet, M3GNET and MACE(you can try small, medium and large variants). We start with a fully working example with CHGnet. Once you worked your way through the netbook and gain some understanding of the mechanics, duplicate the notebook and change it to use your new MLIP.

**HINT** to change the MLIP change the bits of code containing

```python
sp = SinglePoint(
    struct=LFPO.copy(),
    architecture="chgnet",
    device='cuda',
)
```

to 

```python
sp = SinglePoint(
    struct=LFPO.copy(),
    architecture="mace_mp",
    device='cuda',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)
```
for MACE_MP small

or to 

```python
sp = SinglePoint(
    struct=LFPO.copy(),
    architecture="m3gnet",
    device='cuda',
)
```
for M3GNET


this notebook is slightly altered and simplified from https://github.com/materialsvirtuallab/matcalc/blob/main/examples/LiFePO4-NEB.ipynb

codes used
- ASE: https://gitlab.com/ase/ase
- pymatgen: https://github.com/materialsproject/pymatgen
- janus-core: https://github.com/stfc/janus-core

In [ ]:

#packages needed

from ase.io import read,write
from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.geom_opt import optimize
from ase.mep import NEB, NEBTools

from ase.optimize import LBFGS,BFGS
from pymatgen.io.ase import AseAtomsAdaptor
from itertools import chain

from pymatgen.core import PeriodicSite, Structure

from ase.visualize import view

device = "cuda"

# Prepare NEB end structures


## Initial structures

intial structure can be downloaded from materials project, mp-19017, here we we provide the initial structure, the supercell path b and c end structures for convenience.


In [ ]:
LFPO = read("data/LiFePO4_supercell.cif")
view(LFPO,viewer='ngl')

## Relax supercell

In [ ]:
# setup a single point
sp = SinglePoint(
    struct=LFPO.copy(),
    architecture="chgnet",
    device=device)
relaxed_LFPO = optimize(
    struct=sp.struct,
    fmax=0.01)
view(relaxed_LFPO,viewer='ngl')


## Create NEB start, end structures -- b and c directions


In [ ]:
# for end b remove site 11 
# for end c remove site 4
# for start bc remove site 5
# NEB path along b and c directions have the same starting image.

LFPO_end_b = relaxed_LFPO.copy()
del LFPO_end_b[11]

sp_end_b = SinglePoint(
    struct=LFPO_end_b.copy(),
    architecture="chgnet",
    device=device,
)
relaxed_end_b_LFPO = optimize(
    struct=sp_end_b.struct,
    fmax=0.01,
    filter_func=None,
)
view(relaxed_end_b_LFPO,viewer='ngl')


In [ ]:
LFPO_end_c = relaxed_LFPO.copy()
del LFPO_end_c[4]

sp_end_c = SinglePoint(
    struct=LFPO_end_c.copy(),
    architecture="chgnet",
    device=device,
)
relaxed_end_c_LFPO = optimize(
    struct=sp_end_c.struct,
    fmax=0.01,
    filter_func=None,
)
view(relaxed_end_b_LFPO,viewer="ngl")


In [ ]:
LFPO_start_bc = relaxed_LFPO.copy()
del LFPO_start_bc[5]
sp_start_bc = SinglePoint(
    struct=LFPO_start_bc.copy(),
    architecture="chgnet",
    device=device)
relaxed_start_bc_LFPO = optimize(
    struct=sp_start_bc.struct,
    fmax=0.01,
    filter_func=None,
)

view(relaxed_start_bc_LFPO,viewer="ngl")

# Calculate NEB path and barriers

## Path b

In [ ]:
nimages = 7

start = AseAtomsAdaptor.get_structure(relaxed_start_bc_LFPO)
end_b = AseAtomsAdaptor.get_structure(relaxed_end_b_LFPO)
images_p = start.interpolate(end_b, nimages=nimages+1, pbc=False, interpolate_lattices=False, autosort_tol=0.5)
images_b = [ p.to_ase_atoms() for p in images_p]

# Set calculators:
for image in images_b:
    sp = SinglePoint(
        struct=image,
        architecture="chgnet",
        device=device)
neb_b = NEB(images_b,climb=True,allow_shared_calculator=True)

# do the neb ptimize:
opt = BFGS(neb_b, trajectory='neb_b.traj')
opt.run(fmax=0.05)

#view the final path
nebtools_b = NEBTools(images_b)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools_b.get_barrier()

# Get the barrier without any interpolation between highest images.
Ef, dE = nebtools_b.get_barrier(fit=False)

# Get the actual maximum force at this point in the simulation.
max_force = nebtools_b.get_fmax()

# Create a figure like that coming from ASE-GUI.
fig_b = nebtools_b.plot_band()

view(images_b, viewer="ngl")

## Path c

In [ ]:
end_c = AseAtomsAdaptor.get_structure(relaxed_end_c_LFPO)
images_p = start.interpolate(end_c, nimages=nimages+1, pbc=False, interpolate_lattices=False, autosort_tol=0.5)
images_c = [ p.to_ase_atoms() for p in images_p]

# Set calculators:
for image in images_c:
    sp = SinglePoint(
        struct=image,
        architecture="chgnet",
        device=device)
#    print(sp.run()['energy']) 
neb_c = NEB(images_c,climb=True,allow_shared_calculator=True)
# do the nebptimize:
opt = BFGS(neb_c, trajectory='neb_c.traj')
opt.run(fmax=0.05)

#view the final path
nebtools_c = NEBTools(images_c)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools_c.get_barrier()

# Get the barrier without any interpolation between highest images.
Ef, dE = nebtools_c.get_barrier(fit=False)

# Get the actual maximum force at this point in the simulation.
max_force = nebtools_c.get_fmax()

# Create a figure like that coming from ASE-GUI.
fig_c = nebtools_c.plot_band()

view(images_c,viewer="ngl")

# View NEB path in one snapshot

In [ ]:
def generate_snapshot(images: list):
    """Generate a snapshot from images and return an ase atoms."""
    image_structs = list(map(AseAtomsAdaptor().get_structure, images))
    sites = set()
    lattice = image_structs[0].lattice
    for site in chain(*(struct for struct in image_structs)):
        sites.add(PeriodicSite(site.species, site.frac_coords, lattice))
    neb_path = Structure.from_sites(sorted(sites))
    return neb_path.to_ase_atoms()

In [ ]:
# view compressed path b in one snapshot
view(generate_snapshot(images_b),viewer="ngl")

In [ ]:
# view compressed path c in one snapshot
view(generate_snapshot(images_c),viewer="ngl")